# PARTE 1
## Exploración de datos

In [ ]:
%pip install scikit-learn
# necesario porque de otra forma no me deja usar sklearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [6]:
df = pd.read_csv('dataset_pishing.csv')
df

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate
...,...,...
11425,http://www.fontspace.com/category/blackletter,legitimate
11426,http://www.budgetbots.com/server.php/Server%20...,phishing
11427,https://www.facebook.com/Interactive-Televisio...,legitimate
11428,http://www.mypublicdomainpictures.com/,legitimate


In [8]:
df.head()

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


In [7]:
print(df['status'].value_counts())
print(df['status'].value_counts(normalize=True))

status
legitimate    5715
phishing      5715
Name: count, dtype: int64
status
legitimate    0.5
phishing      0.5
Name: proportion, dtype: float64


## Derivación de características
### Atributos del URL a agregar

**Generales**
- length_url - Longitud total de la URL
- num_dots_url - Cantidad de puntos "." en la URL
- num_hyph_url - Cantidad de guiones "-" en la URL
- num_slash_url - Cantidad de barras "/" en la URL
- num_special_chars_url - Total de caracteres especiales (@, ?, =, &, !, %, $, #, +, *, ~)

**Específicas del dominio**
- length_dom - Longitud del dominio
- num_dots_dom - Cantidad de puntos en el dominio (subdominios)
- dom_in_ip - Si el dominio está en formato IP (binaria)
- num_tld_url - Longitud del TLD (top level domain)

**Específicas del pathname**
- length_path - Longitud del path/pathname
- num_hyph_path - Cantidad de guiones en el path

**De parámetros**
- num_param - Número de parámetros (después de "?")
- length_param - Longitud total de los parámetros

**Características de Entropía**
- shannon_entropy - Entropía de Shannon de la URL
- relative_entropy - Entropía relativa (KL divergence)

## Preprocesamiento

In [18]:
label_encoder = LabelEncoder()
df['status'] = label_encoder.fit_transform(df['status'])
# ahora 1 es phishing y 0 es legitimate
print(f"\nDistribución: \n{df['status'].value_counts()}")


Distribución: 
status
0    5715
1    5715
Name: count, dtype: int64
